In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates

# Prepare data

## Read survey and epc data

In [2]:
# Read 'SensorID_EPC_Survey_Open_Data' file
file_path_ID ='Data/SensorID_EPC_Survey_Open_Data.csv'
df_ID_time = pd.read_csv(file_path_ID)

In [3]:
ID_list=df_ID_time['ID']

In [4]:
# Convert field to datetime type 
df_ID_time['Battery Charging Start Time'] = pd.to_datetime(df_ID_time['Battery Charging Start Time'], format="%d/%m/%Y %H:%M")  
df_ID_time['Battery Charging End Time'] = pd.to_datetime(df_ID_time['Battery Charging End Time'], format="%d/%m/%Y %H:%M")  

df_ID_time['Recording Start Time (Participant)'] = pd.to_datetime(df_ID_time['Recording Start Time (Participant) '], format="%d/%m/%Y %H:%M")  
df_ID_time['Recording End Time (Participant)'] = pd.to_datetime(df_ID_time['Recording End Time (Participant) '], format="%d/%m/%Y %H:%M")  

## Define function for heat stress index

In [5]:
# Define function for heat stress index
def calculate_heat_index(temperature, relative_humidity):
    c1 = -42.379
    c2 = 2.04901523
    c3 = 10.14333127
    c4 = -0.22475541
    c5 = -6.83783e-03
    c6 = -5.481717e-02
    c7 = 1.22874e-03
    c8 = 8.5282e-04
    c9 = -1.99e-06

    # Convert temperature to Fahrenheit
    temp_fahrenheit = (temperature * 1.8) + 32

    # Calculate the simple heat index
    heat_index_simple = 0.5 * (temp_fahrenheit + 61.0 + (temp_fahrenheit - 68.0) * 1.2 + (relative_humidity * 0.094))

    if heat_index_simple > 80:
        heat_index = c1 + (c2 * temp_fahrenheit) + (c3 * relative_humidity) + \
                     (c4 * temp_fahrenheit * relative_humidity) + \
                     (c5 * temp_fahrenheit ** 2) + (c6 * relative_humidity ** 2) + \
                     (c7 * temp_fahrenheit ** 2 * relative_humidity) + \
                     (c8 * temp_fahrenheit * relative_humidity ** 2) + \
                     (c9 * temp_fahrenheit ** 2 * relative_humidity ** 2)

        if (relative_humidity < 13) and (80 <= temp_fahrenheit <= 112):
            adjustment = ((13 - relative_humidity) / 4) * ((17 - abs(temp_fahrenheit - 95)) / 17)
            heat_index -= adjustment

        if (relative_humidity > 85) and (80 <= temp_fahrenheit <= 87):
            adjustment = ((relative_humidity - 85) / 10) * ((87 - temp_fahrenheit) / 5)
            heat_index += adjustment

        # Convert temperature to Celsius
        heat_index_celsius = round((heat_index - 32) / 1.8,2)

        return heat_index_celsius
    else:
        heat_index_simple_celsius = round((heat_index_simple - 32) / 1.8,2)
        return heat_index_simple_celsius

# Generate statistics dataframe

In [6]:
description_dfs = []


# Iterate over each sensor
for sensor_id in ID_list:
    try:
        df = pd.read_csv('Data/Household Indoor Sensor Measurement Data/'+sensor_id+'.csv')

        # Convert field to datetime type
        df['TIME'] = pd.to_datetime(df['TIME'])
        df['TIME'] = df['TIME']

        selected_sensor = df_ID_time[df_ID_time['ID'] == sensor_id]
            
        if pd.notnull(selected_sensor['Battery Charging Start Time']).any():
            start_date = selected_sensor['Battery Charging Start Time'] + pd.Timedelta(hours=1)
            # Delay 1 hour to allow the sensor settled in the indoor condition and improve the accuracy of indoor temperature measurements
            end_date = selected_sensor['Battery Charging End Time']
        else:
            start_date = selected_sensor['Recording Start Time (Participant) ']
            end_date = selected_sensor['Recording End Time (Participant) '] 
            
        start_date_value = start_date.iloc[0]
        end_date_value = end_date.iloc[0]

        sliced_df = df[(df['TIME'] >= start_date_value) & (df['TIME'] <= end_date_value)]

        # Extract the date, time, and hour from the timestamp
        sliced_df['Date'] = sliced_df['TIME'].dt.date
        sliced_df['Time'] = sliced_df['TIME'].dt.time
        sliced_df['Hour'] = sliced_df['TIME'].dt.hour

#         # Define a condition to separate daytime (e.g., 6:00 AM to 6:00 PM) and nighttime data
#         daytime_condition = (sliced_df['Hour'] >= 6) & (sliced_df['Hour'] <= 20)

#         # Create separate DataFrames for daytime and nighttime data
#         daytime_data = sliced_df[daytime_condition]
#         nighttime_data = sliced_df[~daytime_condition]

        # # Calculate the maximum daytime temperature and minimum nighttime temperature for each day
        # max_daytime_temperatures = daytime_data.groupby('Date')['TEMP'].max()
        # max_nighttime_temperatures = nighttime_data.groupby('Date')['TEMP'].max()
        # max_daytime_temperatures = max_daytime_temperatures.max()
        # max_nighttime_temperatures = max_nighttime_temperatures.max()
                
        

        # Temperature statistics
        description = sliced_df['TEMP'].describe()
        description_df = pd.DataFrame(description)
        description_df = description_df.T
        description_df.reset_index(drop=True, inplace=True)
        description_df['ID'] = sensor_id

        # # Calculate the maximum daytime temperature and minimum nighttime temperature for each day
        # mean_daytime_temperatures = daytime_data.groupby('Date')['TEMP'].mean()
        # mean_nighttime_temperatures = nighttime_data.groupby('Date')['TEMP'].mean()
        # mean_day = mean_daytime_temperatures.mean()
        # mean_night = mean_nighttime_temperatures.mean()

#         description_df['mean_day'] = mean_day
#         description_df['mean_night'] = mean_night
        
#         description_df['max_day'] = max_daytime_temperatures
#         description_df['max_night'] = max_nighttime_temperatures

        # Add heastress statistics in DataFrame
        sliced_df['heatstress'] = sliced_df.apply(lambda row: calculate_heat_index(row['TEMP'], row['HUM']), axis=1)
        max_heatstress = sliced_df['heatstress'].max()
        min_heatstress = sliced_df['heatstress'].min()
        
        description_df['max_HS'] = max_heatstress
        description_df['min_HS'] = min_heatstress
        
        # Add heat period days in DataFrame
        # Calculate rolling mean over a 3-day window
        rolling_mean = sliced_df['TEMP'].rolling(1440, center=True).mean()
        # Define a threshold temperature for identifying heat periods
        threshold_temp = 25
        # Create a mask to identify heat periods where rolling mean is greater than threshold_temp
        heat_period_mask = rolling_mean > threshold_temp

        heat_periods = []
        current_period = None
        total_heat_period_duration = 0  

        for i, is_heat_period in enumerate(heat_period_mask):
            if is_heat_period:
                if current_period is None:
                    current_period = [i]
                else:
                    current_period.append(i)
            elif current_period is not None:
                if len(current_period) > 4380:  # Check if it's more than 3 days
                    start_date = df['TIME'].iloc[current_period[0]].strftime('%Y-%m-%d')
                    end_date = df['TIME'].iloc[current_period[-1]].strftime('%Y-%m-%d')
                    start_date_timestamp = pd.to_datetime(start_date)
                    end_date_timestamp = pd.to_datetime(end_date)
                    period_duration = (end_date_timestamp - start_date_timestamp).days
                    total_heat_period_duration += period_duration  # Accumulate the duration

                current_period = None

        description_df['heat_period'] = total_heat_period_duration
        
        # Add heat stress index period days in DataFrame
        heatstress_rolling_mean = sliced_df['heatstress'].rolling(1440, center=True).mean()
        # Define a threshold temperature for identifying heat periods
        threshold_temp = 27
        # Create a mask to identify heat periods where rolling mean is greater than threshold_temp
        heatstress_period_mask = heatstress_rolling_mean > threshold_temp
        
        heat_periods = []
        current_period = None
        total_heat_period_duration = 0  # To accumulate the duration of heat periods

        for i, is_heat_period in enumerate(heatstress_period_mask):
            if is_heat_period:
                if current_period is None:
                    current_period = [i]
                else:
                    current_period.append(i)
            elif current_period is not None:
                if len(current_period) > 4380:  # Check if it's more than 3 days
                    start_date = df['TIME'].iloc[current_period[0]].strftime('%Y-%m-%d')
                    end_date = df['TIME'].iloc[current_period[-1]].strftime('%Y-%m-%d')
                    start_date_timestamp = pd.to_datetime(start_date)
                    end_date_timestamp = pd.to_datetime(end_date)
                    period_duration = (end_date_timestamp - start_date_timestamp).days
                    total_heat_period_duration += period_duration  # Accumulate the duration
                current_period = None

        # Heat stress index sustained periods
        description_df['heatstress_period'] = total_heat_period_duration

        # Add monitoring days for each sensor in DataFrame
        periods = sliced_df['Date'].iloc[-1] - sliced_df['Date'].iloc[0]
        periods = periods.days
        description_df['recored_periods'] = periods
        
        # Add relative humidity statistics
        description_df['max_hum'] = sliced_df['HUM'].max()
        
        # Proportion of monitoring time when relative humidity exceed 75%
        hum_over_75 = len(sliced_df[sliced_df['HUM'] > 75])
        per_hum_over_75 = hum_over_75/len(sliced_df)*100
        description_df['perc_hum_75'] = round(per_hum_over_75,2)

        # Days with maximum temperature over 25 degrees    
        daily_max_temp = sliced_df.groupby(sliced_df['Date'])['TEMP'].max()
        days_above_25 = (daily_max_temp > 25).sum()
        description_df['days_above_25'] = days_above_25
        
        # Days with maximum temperature over 27 degrees            
        daily_max_temp = sliced_df.groupby(sliced_df['Date'])['TEMP'].max()
        days_above_27 = (daily_max_temp > 27).sum()
        description_df['days_above_27'] = days_above_27
        
        # Days with maximum temperature over 30 degrees           
        days_above_30 = (daily_max_temp > 30).sum()
        description_df['days_above_30'] = days_above_30

        # Days with maximum temperature over 20 degrees
        days_above_20 = (daily_max_temp > 20).sum()-1 # in case the number are more than monitoring days because of the segmentation
        description_df['days_above_20'] = days_above_20
        
        # During heat wave periods, days with temperature exceeding 27 degree
        heatwave_start_date = pd.to_datetime('2023-09-04').date()
        heatwave_end_date = pd.to_datetime('2023-09-10').date()
        df_hw = sliced_df[(sliced_df['Date'] >= heatwave_start_date) & (sliced_df['Date'] <= heatwave_end_date)]
        daily_max_temp_hw = df_hw.groupby(df_hw['Date'])['TEMP'].max()
        
        days_above_27_hw = (daily_max_temp_hw > 27).sum()
        description_df['hw_days_above_27'] = days_above_27_hw
        
        # During heat wave periods, days with heat stress index exceeding 30 degree        
        days_above_30_hw = (daily_max_temp_hw > 30).sum()
        description_df['hw_days_above_30'] = days_above_30_hw
         
        # During the whole  monitoring days, days with heat stress index exceeding 27 degree   
        daily_max_HS = sliced_df.groupby(sliced_df['Date'])['heatstress'].max()
        HS_days_above_27 = (daily_max_HS > 27).sum()
        description_df['HS_days_above_27'] = HS_days_above_27
        
        # During the whole  monitoring days, days with heat stress index exceeding 32.5 degree    
        HS_days_above_325 = (daily_max_HS >= 32.5).sum()
        description_df['HS_days_above_325'] = HS_days_above_325
       
        # During heat wave periods, days with heat stress index exceeding 27 degree 
        daily_max_HS_hw = df_hw.groupby(df_hw['Date'])['heatstress'].max()
        
        HS_days_above_27_hw = (daily_max_HS_hw > 27).sum()
        description_df['hw_HS_days_above_27'] = HS_days_above_27_hw
        
        # During heat wave periods, days with heat stress index exceeding 32.5 degree 
        HS_days_above_325_hw = (daily_max_HS_hw >= 32.5).sum()
        description_df['hw_HS_days_above_325'] = HS_days_above_325_hw
        
        # Calculate the proportion of monitoring time with temperature over 25 degrees
        time_25=sliced_df[sliced_df['TEMP']>25]['TEMP'].count()
        time_all=sliced_df['TEMP'].count()
        description_df['pro_time_25']=time_25/time_all
        
        # Calculate the max heat stress index on 9th Sep and 10th Sep
        nineth_date = pd.to_datetime('2023-09-09').date()
        tenth_date = pd.to_datetime('2023-09-10').date()
        
        nineth_max_hs=df_hw[df_hw['Date']==nineth_date]['heatstress'].max()
        tenth_max_hs=df_hw[df_hw['Date']==tenth_date]['heatstress'].max()
        
        description_df['9th_HS_max'] = nineth_max_hs
        description_df['10th_HS_max'] = tenth_max_hs
    
        

        # Append the description DataFrame to the list
        description_dfs.append(description_df)
        
    except Exception as e:
        print(f"Error in {str(e)}")

# Concatenate the description DataFrames for all sensors into a single DataFrame
description_df = pd.concat(description_dfs, ignore_index=True)


print(description_df)


C:\Users\ch335s\AppData\Local\Temp\ipykernel_18080\3006003413.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sliced_df['Date'] = sliced_df['TIME'].dt.date
C:\Users\ch335s\AppData\Local\Temp\ipykernel_18080\3006003413.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sliced_df['Time'] = sliced_df['TIME'].dt.time
C:\Users\ch335s\AppData\Local\Temp\ipykernel_18080\3006003413.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

      count       mean       std    min     25%    50%    75%    max  \
0   54906.0  24.414297  2.019142  20.52  23.050  23.97  25.39  30.25   
1   70281.0  25.562214  1.266366  20.54  24.760  25.46  26.32  29.19   
2   56718.0  23.537636  1.763632  20.00  22.160  23.37  24.70  30.70   
3   56919.0  25.728024  1.696041  22.02  24.270  25.77  26.90  30.40   
4   64961.0  25.067441  1.384199  18.98  24.260  25.23  25.83  28.86   
5    1905.0  22.838919  0.794798  21.45  22.160  22.95  23.49  24.27   
6   67838.0  22.689428  1.106300  20.57  21.960  22.53  23.38  25.96   
7   52662.0  23.812827  2.498582  18.27  21.550  23.90  25.56  31.76   
8   55489.0  23.142175  1.767700  18.30  21.840  23.12  24.18  28.12   
9   69039.0  23.306204  2.355669  18.47  21.460  23.10  24.81  31.45   
10  64624.0  24.214287  1.262814  19.94  23.490  24.32  24.98  27.27   
11  50095.0  24.828931  1.462743  21.89  23.510  24.94  25.82  28.07   
12  52417.0  23.534161  2.378468  19.20  21.700  23.45  24.77  2

C:\Users\ch335s\AppData\Local\Temp\ipykernel_18080\3006003413.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sliced_df['heatstress'] = sliced_df.apply(lambda row: calculate_heat_index(row['TEMP'], row['HUM']), axis=1)


In [7]:
pd.set_option('display.max_columns', None)

In [8]:
description_df.head()

,count,mean,std,min,25%,50%,75%,max,ID,max_HS,min_HS,heat_period,heatstress_period,recored_periods,max_hum,perc_hum_75,days_above_25,days_above_27,days_above_30,days_above_20,hw_days_above_27,hw_days_above_30,HS_days_above_27,HS_days_above_325,hw_HS_days_above_27,hw_HS_days_above_325,pro_time_25,9th_HS_max,10th_HS_max
0,54906.0,24.414297,2.019142,20.52,23.05,23.97,25.39,30.25,BL007-02,33.49,20.41,8,6,40,80.33,2.49,19,7,1,40,5,1,9,1,6,1,0.298638,32.12,33.49
1,70281.0,25.562214,1.266366,20.54,24.76,25.46,26.32,29.19,BL007-04,31.92,20.14,29,8,49,71.65,0.00,42,12,0,49,5,0,20,0,6,0,0.669299,31.22,31.82
2,56718.0,23.537636,1.763632,20.00,22.16,23.37,24.70,30.70,BL007-07,33.19,19.63,7,4,41,76.94,0.02,18,7,1,41,5,1,12,1,6,1,0.201347,29.41,29.57
3,56919.0,25.728024,1.696041,22.02,24.27,25.77,26.90,30.40,BL007-09,32.50,21.79,26,9,40,69.69,0.00,30,18,1,40,7,1,22,1,7,1,0.646831,32.50,32.47
4,64961.0,25.067441,1.384199,18.98,24.26,25.23,25.83,28.86,BL007-13,31.07,18.40,18,7,46,74.35,0.00,35,6,0,46,4,0,11,0,5,0,0.555302,30.00,31.07


# Data derived evidence for the TBIJ report

## 1. All sensors achieved 25C

Every single home monitored went over the 25C limit (the World Health Organization maximum recommended indoor temperature for London).

All the sensors achieved a temperature of 25°C during the monitoring periods, except for sensor BL007-14, which recorded temperature data for only 2 days.

In [9]:
description_df[description_df['max']<25]

,count,mean,std,min,25%,50%,75%,max,ID,max_HS,min_HS,heat_period,heatstress_period,recored_periods,max_hum,perc_hum_75,days_above_25,days_above_27,days_above_30,days_above_20,hw_days_above_27,hw_days_above_30,HS_days_above_27,HS_days_above_325,hw_HS_days_above_27,hw_HS_days_above_325,pro_time_25,9th_HS_max,10th_HS_max
5,1905.0,22.838919,0.794798,21.45,22.16,22.95,23.49,24.27,BL007-14,24.4,21.34,0,0,2,65.52,0.0,0,0,0,2,0,0,0,0,0,0,0.0,NaN,NaN


## 2. Proportion of households over 27C and 30C

During the mini-heatwave from 4 to 10 September, 85% of participating households recorded temperatures over 27C. Ten of the homes experienced temperatures over 30C.

In [10]:
count_rows_above_27 =(description_df['hw_days_above_27']!=0).sum()
print(count_rows_above_27)

31


In [11]:
count_rows_heatwave = description_df['9th_HS_max'].notnull().sum()
print(count_rows_heatwave)

37


In [12]:
proportion = round(count_rows_above_27/count_rows_heatwave*100,0)
print(f'{proportion}% of participating households recorded temperatures over 27C')

84.0% of participating households recorded temperatures over 27C


In [13]:
count_rows_above_30 =(description_df['hw_days_above_30']!=0).sum()
print(f'{count_rows_above_30}% of homes experienced temperatures over 30C.')

10% of homes experienced temperatures over 30C.


## 3. Longest sustained heat period

One household saw temperatures remain over the 25C threshold, even at night, for over three weeks. It logged temperatures averaging 27C, despite this not being an especially hot summer. Other homes saw continuous heat periods for 15 and 16 days in a row. 

Please refer to the Jupyter notebook titled 'SustainedHeatPeriod.ipnyb' for more information.

## 4. Indoor and outdoor temperature gap

Some homes were as much as 8-10C hotter at night over the full monitoring period than the outside air temperature, according to readings at local weather stations.

Yunbei

## 5. On 10th Sep, households experienced heat stress index over 27C and 32.5C

The Heat Stress Index takes into account relative humidity as well as temperature and describes how temperatures are likely to be experienced by human bodies. On 10 September, the hottest day of 2023 in England, 25 of the participating homes experienced conditions that felt like 27C (described as ‘Caution’ in the Heat Stress Index), and eight experienced conditions that felt like 32.5C (described as ‘Extreme Caution’ in the Heat Stress Index, with increased risk of heat exhaustion and heatstroke).


In [14]:
tehth_max = description_df[(description_df['10th_HS_max'] >= 27) & (description_df['10th_HS_max'] < 32.5)]
number = len(tehth_max)
print(f'{number} households experienced conditions that felt like 27C')

25 households experienced conditions that felt like 27C


In [15]:
tehth_max = description_df[description_df['10th_HS_max'] >= 32.5]
number = len(tehth_max)
print(f'{number} households experienced conditions that felt like 32.5C')

8 households experienced conditions that felt like 32.5C
